In [1]:
import numpy as np
import os

import pickle
import random
from tqdm import tqdm

In [12]:
import sys
sys.version

'3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) \n[GCC 7.3.0]'

In [2]:
data_folder = '4_2_sim3'

In [3]:
assert data_folder is not None
!rm -r ../data/$data_folder
!gsutil -m cp -r gs://landlord_ai/$data_folder ../data/

Copying gs://landlord_ai/4_2_sim3/0.pkl...
Copying gs://landlord_ai/4_2_sim3/1.pkl...                                      
Copying gs://landlord_ai/4_2_sim3/10.pkl...
Copying gs://landlord_ai/4_2_sim3/11.pkl...                                     
Copying gs://landlord_ai/4_2_sim3/12.pkl...                                     
Copying gs://landlord_ai/4_2_sim3/13.pkl...                                     
Copying gs://landlord_ai/4_2_sim3/14.pkl...
Copying gs://landlord_ai/4_2_sim3/15.pkl...ne                                   
Copying gs://landlord_ai/4_2_sim3/16.pkl...
Copying gs://landlord_ai/4_2_sim3/17.pkl...ne                                   
Copying gs://landlord_ai/4_2_sim3/18.pkl...ne                                   
Copying gs://landlord_ai/4_2_sim3/19.pkl...ne                                   
Copying gs://landlord_ai/4_2_sim3/2.pkl...one                                   
Copying gs://landlord_ai/4_2_sim3/20.pkl...ne                                   
Copying gs://la

In [4]:
directories = ['../data/' + data_folder]

filenames = []
for directory in directories:
    filenames.extend([directory + '/' + file for file in os.listdir(directory)])

In [5]:
all_history_matrices = []
all_move_vectors = []
all_hand_vectors = []
all_y = []

for filename in tqdm(filenames):
    with open(filename, 'rb') as f:
        history_matrices, move_vectors, hand_vectors, y = pickle.load(f)
        
    all_history_matrices.extend(history_matrices)
    all_move_vectors.extend(move_vectors)
    all_hand_vectors.extend(hand_vectors)
    all_y.extend(y)

In [6]:
# shuffle all
feature_limit = 50
            
p = np.random.permutation(len(all_history_matrices))

def densify(sparse_matrix):
    return [x[:feature_limit].todense().astype(np.int8) for x in tqdm(sparse_matrix)]

all_history_matrices = np.array(densify(all_history_matrices), dtype=np.int8)[p]

100%|██████████| 4583979/4583979 [09:21<00:00, 8160.46it/s] 


TypeError: only integer scalar arrays can be converted to a scalar index

In [7]:
all_move_vectors = np.array(all_move_vectors)[p]
all_hand_vectors = np.array(all_hand_vectors)[p]
all_y = np.array(all_y)[p]

In [8]:
from sklearn.model_selection import train_test_split
train_hm, test_hm, train_mv, test_mv, train_hv, test_hv, train_y, test_y = \
    train_test_split(all_history_matrices, all_move_vectors, all_hand_vectors, all_y, test_size=0.1)

In [9]:
!rm -r ../data/{data_folder}_agg/
!mkdir ../data/{data_folder}_agg/

In [17]:
with open('../data/' + data_folder + '_agg/train.pkl', 'wb') as f:
    pickle.dump((train_hm, train_mv, train_hv, train_y), f, protocol=4)

with open('../data/' + data_folder + '_agg/test.pkl', 'wb') as f:
    pickle.dump((test_hm, test_mv, test_hv, test_y), f, protocol=4)

In [18]:
!gsutil cp -r ../data/{data_folder}_agg/ gs://landlord_ai/

Copying file://../data/4_2_sim3_agg/train.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://../data/4_2_sim3_agg/test.pkl [Content-Type=application/octet-stream]...
| [2 files][  5.9 GiB/  5.9 GiB]  103.2 MiB/s                                   
Operation completed over 2 objects/5.9 GiB.             